In [10]:
import numpy as np # Pour la manipulation de tableaux

import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

from tensorflow.keras.layers import Input, Dense #Pour instancier une couche Dense et une d'Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

In [47]:
df = pd.read_csv('df_final_rolling_ready_50_with_bookodds.csv')
df

,year,month,day,Series,Court,Surface,prize_money_euro,Round,Best of,player_1_B365,...,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_return_games_played,player_2_service_points_won,player_2_service_points_total,player_2_return_points_won,player_2_return_points_total,player_2_under_pressure_rating,player_2_win_rate
0,2004,1,5,International,Outdoor,Hard,850000,1st Round,3,1.364,...,34.276596,0.358473,8.659574,12.276596,0.586849,80.617021,0.395196,81.063830,125.323158,0.489362
1,2004,1,5,International,Outdoor,Hard,323000,1st Round,3,1.571,...,22.000000,0.385354,5.200000,10.800000,0.667952,66.000000,0.345974,65.800000,104.630592,0.400000
2,2004,1,5,International,Outdoor,Hard,323000,1st Round,3,1.727,...,31.481481,0.475515,7.111111,11.962963,0.571163,79.888889,0.403513,75.740741,122.963656,0.444444
3,2004,1,5,International,Outdoor,Hard,323000,1st Round,3,1.727,...,35.533333,0.539870,6.533333,14.600000,0.614859,95.466667,0.381666,92.533333,162.917915,0.533333
4,2004,1,5,International,Outdoor,Hard,323000,1st Round,3,1.727,...,35.125000,0.487531,5.625000,13.312500,0.617745,84.750000,0.329245,86.312500,145.084898,0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28661,2019,11,13,Masters Cup,Indoor,Hard,7650000,Round Robin,3,2.000,...,34.180000,0.414256,9.180000,13.640000,0.642209,87.440000,0.391307,90.880000,150.190420,0.600000
28662,2019,11,14,Masters Cup,Indoor,Hard,7650000,Round Robin,3,1.360,...,33.560000,0.451182,7.760000,12.840000,0.698233,79.460000,0.395017,85.760000,159.101521,0.780000
28663,2019,11,16,Masters Cup,Indoor,Hard,7650000,Semifinals,3,3.500,...,33.280000,0.452253,7.740000,12.820000,0.701363,78.760000,0.395938,85.340000,157.875330,0.780000
28664,2019,11,16,Masters Cup,Indoor,Hard,7650000,Semifinals,3,1.800,...,33.840000,0.399256,9.040000,13.560000,0.640075,86.920000,0.385923,90.240000,149.166610,0.580000


In [48]:
# Encodage
le = LabelEncoder()
le.fit(pd.concat([df['player_1_name'], df['player_2_name']], axis=0))
df['player_1_name_encoded'] = le.transform(df['player_1_name'])
df['player_2_name_encoded'] = le.transform(df['player_2_name'])
# Suppression player_1_name & player_2
df.drop(['player_1_name', 'player_2_name'], axis=1, inplace=True)
# Dichotomisation
df = pd.get_dummies(df)
# Split
nb_rows_train = int(round(len(df)*0.7,0))
X_train = df.drop('player_1_win', axis=1)[:nb_rows_train]
y_train = df['player_1_win'][:nb_rows_train]
X_test = df.drop('player_1_win', axis=1)[nb_rows_train:]
y_test = df['player_1_win'][nb_rows_train:]
# Standardisation
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
inputs = Input(shape = (123))

dense1 = Dense(units = 8, activation = "relu",kernel_initializer ='normal', name = "Dense_1")

dense2 = Dense(units = 4, activation = "relu",kernel_initializer ='normal', name = "Dense_2")
dense3 = Dense(units=2, activation='softmax',kernel_initializer ='normal', name = "Dense_3")
x=dense1(inputs)
x=dense2(x)

outputs=dense3(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 123)]             0         
                                                                 
 Dense_1 (Dense)             (None, 8)                 992       
                                                                 
 Dense_2 (Dense)             (None, 4)                 36        
                                                                 
 Dense_3 (Dense)             (None, 2)                 10        
                                                                 
Total params: 1,038
Trainable params: 1,038
Non-trainable params: 0
_________________________________________________________________


In [50]:
es = EarlyStopping(monitor='val_accuracy', mode='min', patience=10,min_delta=0)

In [51]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

In [52]:
X_train.shape[1]

123

In [53]:
model.fit(X_train_scaled,y_train,epochs=30,batch_size=32,validation_split=0.15)

y_pred = model.predict(X_test_scaled)

Epoch 1/30
533/533 [==============================] - 2s 2ms/step - loss: 0.6207 - accuracy: 0.6492 - val_loss: 0.5857 - val_accuracy: 0.6731
Epoch 2/30
533/533 [==============================] - 1s 2ms/step - loss: 0.5768 - accuracy: 0.6911 - val_loss: 0.5820 - val_accuracy: 0.6824
Epoch 3/30
533/533 [==============================] - 1s 2ms/step - loss: 0.5700 - accuracy: 0.6964 - val_loss: 0.5789 - val_accuracy: 0.6917
Epoch 4/30
533/533 [==============================] - 1s 2ms/step - loss: 0.5676 - accuracy: 0.7016 - val_loss: 0.5784 - val_accuracy: 0.6944
Epoch 5/30
533/533 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7017 - val_loss: 0.5777 - val_accuracy: 0.6907
Epoch 6/30
533/533 [==============================] - 2s 3ms/step - loss: 0.5647 - accuracy: 0.7019 - val_loss: 0.5787 - val_accuracy: 0.6884
Epoch 7/30
533/533 [==============================] - 1s 2ms/step - loss: 0.5635 - accuracy: 0.7042 - val_loss: 0.5810 - val_accuracy: 0.6864
Epoch 

In [45]:
y_pred = np.argmax(y_pred, axis=1)

In [46]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.69      0.67      4351
           1       0.66      0.62      0.64      4249

    accuracy                           0.65      8600
   macro avg       0.65      0.65      0.65      8600
weighted avg       0.65      0.65      0.65      8600

[[2998 1353]
 [1620 2629]]


In [53]:
X_train.shape[1]

119